In [4]:
# tell Python to use the ecco_v4_py in the 'ECCOv4-py' repository
from os.path import join,expanduser
import sys

# identify user's home directory
user_home_dir = expanduser('~')

# import the ECCOv4 py library 
sys.path.insert(0,join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco
from ecco_v4_py import vector_calc, scalar_calc

import gsw
import cmocean
from collections import Counter
from dask.distributed import Client
import datetime
import json
import numpy as np
from pathlib import Path
from pprint import pprint
import sys
import time as time
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.axes as ax

# Load Function

In [5]:
def calc_meridional_stf_dens(ds, lat_vals, sig_levels,
                             basin_name=None):
    """
    Compute meridonal streamfunction at each density level defined in sig_levels
    across latitude(s) defined in lat_vals

    Parameters
    ----------
    ds : xarray DataSet
        must contain vars UVELMASS,VVELMASS, UVELSTAR, VVELSTAR, SIG2, drF, dyG, dxG
        and coords YC, (maskW, and maskS if basin_name not None)
    lat_vals : float or list
        latitude value(s) specifying where to compute the streamfunction
    sig_levels : list of length 50
        Target values of Sigma_2 specifying the density bands for the 
        computation of the streamfunction
    basin_name : string, optional
        denote ocean basin over which to compute streamfunction
        If not specified, compute global quantity
        see get_basin.get_available_basin_names for options

    Returns
    -------
    xds : xarray Dataset
        with the main variable
            'psi'
                meridional overturning streamfunction across the section in Sv
                with dimensions 'time' (if in given dataset), 'lat', and 'SIGMA_levs'
    """
    #Set up vars
    #velocities
    U = ds.UVELMASS
    u = ds.UVELSTAR
    V = ds.VVELMASS
    v = ds.VVELSTAR
    utot = U+u
    vtot = V+v
    #spacial
    dx = ds.dxG
    dy = ds.dyG
    dz = ds.drF
    sig2 = ds.SIG2
    
    #regrid
    grid = ecco.get_llc_grid(ds)
    
    sig2W = grid.interp(sig2, axis='X')
    sig2S = grid.interp(sig2, axis='Y')

    #basin mask?
    if basin_name is not None:
        maskS = ds_geom.maskS.compute()
        maskW = ds_geom.maskW.compute()
        basin_maskW = ecco.get_basin_mask(basin_name= basin_name,mask=maskW.isel(k=0))
        basin_maskS = ecco.get_basin_mask(basin_name= basin_name,mask=maskS.isel(k=0))
        ubasin = utot*basin_maskW
        vbasin = vtot*basin_maskS
        sigWbasin = sig2W*basin_maskW
        sigSbasin = sig2S*basin_maskS
        u = ubasin
        v = vbasin
        sigW = sigWbasin
        sigS = sigSbasin
    else:
        u = utot
        v = vtot
        sigW = sig2W
        sigS = sig2S

    #compute streamfunction everywhere, summing over all densities greater than target density
    xvol = xr.zeros_like(u)
    xvol = xvol.rename({'k':'sig2'})
    yvol = xr.zeros_like(v)
    yvol = yvol.rename({'k':'sig2'})
    for ss in range(50):
        sig = sig_levels[ss]
        y = v*dz*dx*np.heaviside(sigS-sig,1)*-1
        yy = y.sum(dim='k')
        x = utot*dz*dy*np.heaviside(sigW-sig,1)*-1
        xx = x.sum(dim='k')
        if any(x in ['time','month','year'] for x in list(U.dims)):
            yvol[:,ss,:] = yy
            xvol[:,ss,:] = xx
        else:
            yvol[ss,:] = yy
            xvol[ss,:] = xx
        
    #now compute meridional streamfunction
    # Initialize empty DataArray with coordinates and dims
    ones = xr.ones_like(ds.YC)
    
    lats_da = xr.DataArray(lat_vals,coords={'lat':lats},dims=('lat',))
    
    xda = xr.zeros_like(yvol['sig2']*lats_da)

    if 'time' in list(U.dims):
        xda = xda.broadcast_like(yvol['time']).copy()
    elif 'month' in list(U.dims):
        xda = xda.broadcast_like(yvol['month']).copy()
    elif 'year' in list(U.dims):
        xda = xda.broadcast_like(yvol['year']).copy()
    
    # Convert to dataset to add sigma2 coordinate
    xds = xda.to_dataset(name='psi')
    xds = xds.assign_coords({'SIGMA_levs':('sig2', sig_levels)})
    
    #cycle through all lats
    for l in range(len(lat_vals)):
        lat = lat_vals[l]
        dome_maskC = ones.where(ds.YC>=lat,0).compute()
        lat_maskW = grid.diff(dome_maskC,'X',boundary='fill') #multiply by x
        lat_maskS = grid.diff(dome_maskC,'Y',boundary='fill') #multiply by y
        ytrsp_lat = (yvol * lat_maskS).sum(dim=['i','j_g','tile'])
        xtrsp_lat = (xvol * lat_maskW).sum(dim=['i_g','j','tile'])
        xds['psi'].loc[{'lat':lat}] = ytrsp_lat+xtrsp_lat

    return xds

## download data

In [6]:
ecco_v4r5_mon_mean_native_dir = Path('/efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/')

# list sub-directories (one per dataset)
ecco_v4r5_mon_mean_native_dataset_paths = np.sort(list(ecco_v4r5_mon_mean_native_dir.glob('*')))

# Select a dataset (the one containing temperature and salinity)
dataset_num = 22

print('selected ', ecco_v4r5_mon_mean_native_dataset_paths[dataset_num])
Vel_dataset_dir = ecco_v4r5_mon_mean_native_dataset_paths[dataset_num]

# make a list of all of the files in the directory
Vel_dataset_files = np.sort(list(Vel_dataset_dir.glob('*nc')))

# show first 5 files
print('\nFirst 5 files')
pprint([x.name for x in Vel_dataset_files[:5]])

# Select a dataset (the one containing temperature and salinity)
dataset_num = 14

print('selected ', ecco_v4r5_mon_mean_native_dataset_paths[dataset_num])
dens_dataset_dir = ecco_v4r5_mon_mean_native_dataset_paths[dataset_num]

# make a list of all of the files in the directory
dens_dataset_files = np.sort(list(dens_dataset_dir.glob('*nc')))

# show first 5 files
print('\nFirst 5 files')
pprint([x.name for x in dens_dataset_files[:5]])

selected  /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_VOLUME_FLUX

First 5 files
['OCEAN_VOLUME_FLUX_mon_mean_1992-01_ECCO_V4r5_native_llc0090.nc',
 'OCEAN_VOLUME_FLUX_mon_mean_1992-02_ECCO_V4r5_native_llc0090.nc',
 'OCEAN_VOLUME_FLUX_mon_mean_1992-03_ECCO_V4r5_native_llc0090.nc',
 'OCEAN_VOLUME_FLUX_mon_mean_1992-04_ECCO_V4r5_native_llc0090.nc',
 'OCEAN_VOLUME_FLUX_mon_mean_1992-05_ECCO_V4r5_native_llc0090.nc']
selected  /efs_ecco/ECCO/V4/r5/netcdf/native/mon_mean/OCEAN_BOLUS_VELOCITY

First 5 files
['OCEAN_BOLUS_VELOCITY_mon_mean_1992-01_ECCO_V4r5_native_llc0090.nc',
 'OCEAN_BOLUS_VELOCITY_mon_mean_1992-02_ECCO_V4r5_native_llc0090.nc',
 'OCEAN_BOLUS_VELOCITY_mon_mean_1992-03_ECCO_V4r5_native_llc0090.nc',
 'OCEAN_BOLUS_VELOCITY_mon_mean_1992-04_ECCO_V4r5_native_llc0090.nc',
 'OCEAN_BOLUS_VELOCITY_mon_mean_1992-05_ECCO_V4r5_native_llc0090.nc']


In [8]:
from dask.distributed import Client

#  connec to existing LocalCluster
# the port number will be different!
client = Client("tcp://127.0.0.1:39295") #find within Scheduler Address with orange red squares)
client.ncores
client.restart()

In [9]:
start_time = time.time();
ds_bolus = None
ds_vel = None

files_to_load = Vel_dataset_files

print(f'lazy-loading {len(files_to_load)} granules')
# first lazy load
ds_vel = xr.open_mfdataset(Vel_dataset_files, 
                          parallel=True, data_vars='minimal',\
                          coords='minimal',compat='override',
                          combine='nested', concat_dim='time',
                          chunks={'time':12, 'tile':13,' k':50,'j':90,'i':90})
files_to_load = dens_dataset_files

print(f'lazy-loading {len(files_to_load)} granules')
# first lazy load
ds_bolus = xr.open_mfdataset(dens_dataset_files, 
                          parallel=True, data_vars='minimal',\
                          coords='minimal',compat='override',
                          combine='nested', concat_dim='time',
                          chunks={'time':12, 'tile':13,' k':50,'j':90,'i':90})


total_time = time.time() - start_time

lazy-loading 336 granules
lazy-loading 336 granules


In [10]:
geom_path = '/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc'
ds_geom = xr.open_dataset(geom_path)

In [11]:
sig_path = '/efs_ecco/czimmerm/Sigma_2_field.nc'
ds_sig = xr.open_dataset(sig_path,\
                         chunks={'time':1, 'tile':13,' k':25,'j':90,'i':90})

## Reformat

In [13]:
ds = xr.merge((ds_geom , ds_bolus[['VVELSTAR','UVELSTAR']],ds_vel[['UVELMASS','VVELMASS']],ds_sig['SIG2']))
#ds = ds.load()

## Customize the Streamfunction you want to calculate

In [15]:
#take climatology or time mean or select specific time window
#clim
ds_clim = ds.groupby('time.month').mean('time')
#mean
ds_mean = ds.mean('time')
#annual mean
ds_an_mean = ds.groupby('time.year').mean('time')
#window
ds_years = ds.isel(time = list(np.arange(192,228,1))) #this window is 01-2008 until 12-2010

In [18]:
#pick latitudes
#lats = np.arange(-80,80,1) #global
lats = np.arange(-30,80,1) #atlantic or indo-pacific
#pick basin -- default is None
basin_name = 'atlExt' 

In [ ]:
#pick target sigma levels --> define new coordinates
#first look at your real sig values
sig = ds_sig.SIG2.mean('time')
sig_section = []
for k in range(50):
    nlon_centers, nlat_centers, nlone, nlate, sig_reg = \
        ecco.resample_to_latlon(sig.XC, sig.YC, 
                                sig[k], -90, 90,
                                0.2, -180, 180, 0.2, 
                                radius_of_influence=200000.0,
                                mapping_method='nearest_neighbor')
    sig_section.append(sig_reg)
sig_section= np.array(sig_section)
np.shape(sig_section)

In [ ]:
#pick a lat to view
j_SO = np.argmax(nlat_centers[:,0] >= -60) #Southern ocean
print(nlat_centers[j_SO,0])
#view
plt.pcolor(nlon_centers[0,:], sig.Z, sig_section[:, j_SO,:])
cb=plt.colorbar()

In [ ]:
#pick a lon to view
i_atl = np.argmax(nlon_centers[0,:] >= -30) #atlantic
print(nlon_centers[0,i_atl])
#view
plt.pcolor(nlat_centers[:,0], sig.Z, sig_section[:, :,i_atl])
cb=plt.colorbar()

In [17]:
# now fill in appropriate target sigs based on location of interest -- MUST BE LENGTH 50
target_sig_levels = [28.5200, 29.4855, 30.8803, 31.6772, 32.1183,32.8800, 33.2711,33.8000, 34.0234,34.2500, 34.5587,34.800,35.1000, 35.3480,35.4700,35.6100, 35.7690,35.8600,35.9700, 36.0550, 36.1500, 36.2469, 36.2900, 36.3810,36.4300,36.4512, 36.4893,36.5100, 36.5964,36.6500, 36.7148, 36.7700, 36.8421, 36.9200, 36.9887,37.0100,37.0323, 37.0598,37.1000, 37.1254, 37.1400,37.1612,37.1840,37.2024, 37.2346, 37.2780, 37.3160, 37.3498,37.3672, 37.3806];
len(target_sig_levels) #must be 50

50

## Run the function

In [ ]:
#fill in whichever ds, lats, and sig levels you decide on
strm = calc_meridional_stf_dens(ds_clim, lats, target_sig_levels, basin_name)# remove basin name if global

/srv/conda/envs/notebook/lib/python3.12/site-packages/xgcm/grid_ufunc.py:832: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg
/srv/conda/envs/notebook/lib/python3.12/site-packages/xgcm/grid_ufunc.py:832: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


get_basin_name:  ['atl', 'mexico', 'hudson', 'med', 'north', 'baffin', 'gin'] /home/jovyan/ECCOv4-py/binary_data
load_binary_array: loading file /home/jovyan/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
shape after reading 
(13, 90, 90)
get_basin_name:  ['atl', 'mexico', 'hudson', 'med', 'north', 'baffin', 'gin'] /home/jovyan/ECCOv4-py/binary_data
load_binary_array: loading file /home/jovyan/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
shape after reading 
(13

## Visualize output

### timeseries

In [ ]:
#get max strmfxn at each latitude
Psi_moc = strm['psi'].max(dim='sig2')
#plot timeseries for some latitude
lat_plot = 50 #N
lat_indx = np.argmax(strm['lat'].values >= lat_plot)

#if climatology
plt.plot(strm['month'],Psi_moc[:,lat_indx]*10**-6)
plt.title(f'Seasonal cycle of maximum [global/atlantic] Meridional Streamfunction at {lat_plot}N')
plt.ylabel('Psi [SV]')
plt.xlabel('Month of year') # 1 = January
# #if annual mean
# plt.plot(strm['year'],Psi_moc[:,lat_indx]*10**-6)
# plt.title(f'Annual mean Maximum [global/atlantic] Meridional Streamfunction at {lat_plot}N')
# plt.ylabel('Psi [SV]')
# plt.xlabel('year') # 
# #if full or time window
# plt.plot(strm['time'],Psi_moc[:,lat_indx]*10**-6)
# plt.title(f'Monthly Maximum [global/atlantic] Meridional Streamfunction at {lat_plot}N')
# plt.ylabel('Psi [SV]')
# plt.xlabel('time') # 

### density space plots

In [ ]:
#take time mean if desired
Psi_time_mean = strm['psi'].mean(dim='month') #or month, year ...

In [ ]:
#plot latitude/density map
#centeres colorbar at 0, remove if desired
maxlev = 36
deltlev = 3
clevs = np.concatenate((np.arange(-1*maxlev,0,deltlev),np.arange(deltlev,maxlev+deltlev/2)))

fig, ax = plt.subplots()
#plot time mean
plt.contourf(strm.lat,strm.SIGMA_levs,Psi_time_mean*10**-6,levels = clevs,cmap='PiYG')
#plot specific time (first time selected here)
#plt.contourf(strm.lat,strm.SIGMA_levs,strm.psi[0]*10**-6,levels = clevs,cmap='PiYG')

cb=plt.colorbar()
cb.set_label('Sv')
plt.title('Time Mean Atlantic Meridional Streamfunction')
plt.ylabel('density (sig_2)')
plt.xlabel('lat')
plt.gca().invert_yaxis() #have increasing density down the y-axis
plt.show()

In [ ]:
#plot latitude/density map squishing the y axis

fig, ax = plt.subplots()
#plot time mean
plt.contourf(strm.lat,np.arange(0,50),Psi_time_mean*10**-6)
#plot specific time (first time selected here)
#plt.contourf(strm.lat,strm.SIGMA_levs,strm.psi[0]*10**-6,levels = clevs,cmap='PiYG')

cb=plt.colorbar()
cb.set_label('Sv')
plt.title('Time Mean [Global/ Atlantic] Meridional Streamfunction')
plt.ylabel('density (sig_2)')
plt.xlabel('lat')
plt.gca().invert_yaxis() #have increasing density down the y-axis

sig_4_plot = ds_strm.SIGMA_levs.values[np.arange(0,50,5)]
sig_4_plot
ax.yaxis.set_ticks(np.arange(0,50,5))
ax.set_yticklabels(sig_4_plot)

plt.show()

In [ ]:
#plot profile at a single latitude
lat_plot = 50 #N
lat_indx = np.argmax(strm['lat'].values >= lat_plot)

plt.figure(figsize=(12,6))
#plot time mean
plt.plot(Psi_time_mean[:,lat_indx]*10**-6,strm['SIGMA_levs'],'r')
#plot specific time
#plt.plot(strm.psi[0,:,lat_indx]*10**-6,strm['SIGMA_levs'],'r')

plt.grid()
plt.title(f'Meridional Streamfunction at {lat_plot}N')
plt.ylabel('sig')
plt.xlabel('Sv')
plt.ylim([strm['SIGMA_levs'][-1],strm['SIGMA_levs'][2]])#adjust axis to highlight denser density classes
#plt.gca().invert_yaxis() #invert axis if not adjusting limits
plt.show()